In [328]:
import numpy as np
from collections import OrderedDict
import time
import pprint
"""
(self, *, wavelength = 1550, n_0 = 1.45, n_1 = 1.60, n_s = 1.45, a = 4.0e-6,
                 d = 4.0e-6,p = 4, q = 4, Power = 10, step_size = 1):
"""

class planar_waveguide:

    def __init__(self, *, wavelength = 1550, n_0 = 1.45, n_1 = 1.60, n_s = 1.45, a = 4.0e-6,
                 d = 4.0e-6,p = 3, q = 4, Power = 10, step_size = 1):
        
        self.wavelength = wavelength*1e-9                  #Wavelength of the light
        self.a = a                                         #Half width of the waveguide (x axis)
        self.d = d                                         #Half depth of the waveguide (y axis)
        self.p = p                                         #Mode number on x axis
        self.q = q                                         #Mode number on y axis
        self.n_0 = n_0                                     #Refractive index of the cladding
        self.n_1 = n_1                                     #Refractive index of the core
        self.n_s = n_s                                     #Refractive index of the substrate (default: n_s = n_0, otherwise n_0<=n_s<n_1)
        self.Power = Power                                 #Laser Power in Watts
        self.step_size = step_size                         #Step size for the iterations
        self.epsilon_0 = 8.854e-12                         #Permittivity of free space
        self.mu_0 = 4*np.pi*1e-7                           #Permeability of free space
        self.c = 1/np.sqrt(self.epsilon_0 * self.mu_0)     #Speed of light in m/s
        
    def delta(self):
        """
        rdelta = Relative refractive index difference
        n_1 = Refractive index of the core
        n_s = Refractive index of the cladding, this should be higher than n_0 but lower than n_1
        """
        rdelta = (self.n_1**2 - self.n_s**2) / (2 * self.n_1**2)
        return rdelta

    def frequency(self):
        """
        Determines the wave frequency
        """
        freq = self.c / self.wavelength
        return freq
    
    def ang_freq(self):
        """
        rang_freq = Returns the Angular frequency
        """
        rang_freq = self.frequency() * np.pi * 2
        return rang_freq
    
        
    def k(self):
        """
        rk = Wavenumber in vacuum
        """
        rk = self.ang_freq() / self.c 
        return rk
        
    def va(self):
        """
        rva = Normalized frequency using 'a' dimension
        """
        rva = self.k() * self.n_1 * self.a * np.sqrt(2 * self.delta())
        return rva
    
    def vd(self):
        """
        rvd = Normalized frequency using 'd' dimension
        """
        rvd = self.k() * self.n_1 * self.d * np.sqrt(2 * self.delta())
        return rvd

    def gamma(self):
        """
        rgamma = Refractive index asymmetry factor
        """
        rgamma = (self.n_s**2 - self.n_0**2) / (self.n_1**2 - self.n_s**2)
        return rgamma
        
    def ba(self):
        """
        results = Dictionary with keys = mode number and values = normalized propagation constant b. This is using the 
        a dimension and p values starting from 0 to p-1
        """
        matrix = np.zeros((1,int(self.step_size*1e6)))
        range_vector = np.resize(np.linspace(0,0.9999999999,self.step_size*1e6),(1,self.step_size*1e6))
        matrix +=range_vector
        p_vector = np.resize(np.arange(0,self.p,1),(self.p,1))
        aa = 2 * self.va() * np.sqrt(1-matrix)
        bb = p_vector * np.pi + np.arctan(np.sqrt(matrix/(1-matrix)))+np.arctan(np.sqrt((matrix+self.gamma())/(1-matrix)))
        matrix2 = aa-bb
        results = OrderedDict()
        for elements in p_vector:
            min_val = np.argmin(np.absolute(matrix2[elements]))
            result = matrix[0][min_val]
            results[int(elements)] = result
        return results
    
    def bd(self):
        """
        results = Dictionary with keys = mode number and values = normalized propagation constant b. This is using the 
        d dimension and q values starting from 0 to q-1
        """
        matrix = np.zeros((1,int(self.step_size*1e6)))
        range_vector = np.resize(np.linspace(0,0.9999999999,self.step_size*1e6),(1,self.step_size*1e6))
        matrix +=range_vector
        q_vector = np.resize(np.arange(0,self.q,1),(self.q,1))
        aa = 2 * self.vd() * np.sqrt(1-matrix)
        bb = q_vector * np.pi + np.arctan(np.sqrt(matrix/(1-matrix)))+np.arctan(np.sqrt((matrix+self.gamma())/(1-matrix)))
        matrix2 = aa-bb
        results = OrderedDict()
        for elements in q_vector:
            min_val = np.argmin(np.absolute(matrix2[elements]))
            result = matrix[0][min_val]
            results[int(elements)] = result
        return results
   
    def n_ea(self):
        """
        Calculates effective refractive index using the a dimension
        """
        n_e = OrderedDict()
        for p,b in self.ba().items():
            effective_index = np.sqrt(b * (self.n_1**2 - self.n_0**2) + self.n_s**2)
            n_e[p] = effective_index
        return n_e
        
    def n_ed(self):
        """
        Calculates effective refractive index using the d dimension
        """
        n_e = OrderedDict()
        for q,b in self.bd().items():
            effective_index = np.sqrt(b * (self.n_1**2 - self.n_0**2) + self.n_s**2)
            n_e[q] = effective_index
        return n_e
        
    def betaa(self):
        """
        Calculates beta (propagation constant along z axis) using the a dimension
        """
        beta = OrderedDict()
        for p,elements in self.n_ea().items():
            beta[p] = elements * self.k()
        return beta
        
    def betad(self):
        """
        Calculates beta (propagation constant along z axis) using the d dimension
        """
        beta = OrderedDict()
        for q,elements in self.n_ed().items():
            beta[q] = elements * self.k()
        return beta
        
    def norm_wavenuma(self):
        """
        Normalized wavenumbers on the a dimension
        """
        wavenumbers = OrderedDict()
        for p,b in self.ba().items():
            u = self.va() * np.sqrt(1-b)
            w = self.va() * np.sqrt(b)
            w_prime = self.va() * np.sqrt(b + self.gamma())
            phi = p * np.pi * 0.5 +0.5 * np.arctan(w/u) + 0.5 * np.arctan(w_prime/u)
            wavenumbers[p] = {'u':u, 'w':w, 'w_prime':w_prime, 'phi':phi}
        return wavenumbers
    
    def norm_wavenumd(self):
        """
        Normalized wavenumbers on the d dimension
        """
        wavenumbers = OrderedDict()
        for q,b in self.bd().items():
            u = self.vd() * np.sqrt(1-b)
            w = self.vd() * np.sqrt(b)
            w_prime = self.vd() * np.sqrt(b + self.gamma())
            phi = q * np.pi * 0.5 +0.5 * np.arctan(w/u) + 0.5 * np.arctan(w_prime/u)
            wavenumbers[q] = {'u':u, 'w':w, 'w_prime':w_prime, 'phi':phi}
        return wavenumbers
    
    def Aa(self):
        """
        Calculates the amplitude in the a dimension
        """
        A = OrderedDict()
        for p,wavenumbers in self.norm_wavenuma().items():
            a = 2 * self.ang_freq() * self.mu_0 * self.Power
            b = self.betaa()[p] * self.a * (1+1/(2*self.norm_wavenuma()[p]['w'])+1/(2*self.norm_wavenuma()[p]['w_prime']))
            amplitude = np.sqrt(a/b)
            A[p] = amplitude
        return A
    
    def Ad(self):
        """
        Calculates the amplitude in the d dimension
        """
        A = OrderedDict()
        for q,wavenumbers in self.norm_wavenumd().items():
            a = 2 * self.ang_freq() * self.mu_0 * self.Power
            b = self.betad()[q] * self.d * (1+1/(2*self.norm_wavenumd()[q]['w'])+1/(2*self.norm_wavenumd()[q]['w_prime']))
            amplitude = np.sqrt(a/b)
            A[q] = amplitude
        return A
    
    def Power_dista(self):
        """
        Calculates the power distribution on the a dimension. P_clad is the power in the cladding, P_core is the power 
        in the core and P_sub is the power in the substrate 
        """
        Power_dist = OrderedDict()
        for p, elements in self.Aa().items():
            a = self.betaa()[p] * self.a * elements**2
            b = 2 * self.ang_freq() * self.mu_0
            c = (np.sin(self.norm_wavenuma()[p]['u']+self.norm_wavenuma()[p]['phi']))**2
            d = (np.sin(self.norm_wavenuma()[p]['u']-self.norm_wavenuma()[p]['phi']))**2
            e =(np.cos(self.norm_wavenuma()[p]['u']+self.norm_wavenuma()[p]['phi']))**2
            f =(np.cos(self.norm_wavenuma()[p]['u']-self.norm_wavenuma()[p]['phi']))**2
            P_core = (a/b) * (1+c/(2*self.norm_wavenuma()[p]['w'])+d/(2*self.norm_wavenuma()[p]['w_prime']))
            P_sub = (a/b) * (e/(2*self.norm_wavenuma()[p]['w']))
            P_clad = (a/b) * (f/(2*self.norm_wavenuma()[p]['w_prime']))
            Power_dist[p] = {'P_core':P_core, 'P_sub':P_sub, 'P_clad':P_clad}
        return Power_dist
    
    def Power_distd(self):
        """
        Calculates the power distribution on the d dimension. P_clad is the power in the cladding, P_core is the power 
        in the core and P_sub is the power in the substrate 
        """
        Power_dist = OrderedDict()
        for q, elements in self.Ad().items():
            a = self.betad()[q] * self.d * elements**2
            b = 2 * self.ang_freq() * self.mu_0
            c = (np.sin(self.norm_wavenumd()[q]['u']+self.norm_wavenumd()[q]['phi']))**2
            d = (np.sin(self.norm_wavenumd()[q]['u']-self.norm_wavenumd()[q]['phi']))**2
            e =(np.cos(self.norm_wavenumd()[q]['u']+self.norm_wavenumd()[q]['phi']))**2
            f =(np.cos(self.norm_wavenumd()[q]['u']-self.norm_wavenumd()[q]['phi']))**2
            P_core = (a/b) * (1+c/(2*self.norm_wavenumd()[q]['w'])+d/(2*self.norm_wavenumd()[q]['w_prime']))
            P_sub = (a/b) * (e/(2*self.norm_wavenumd()[q]['w']))
            P_clad = (a/b) * (f/(2*self.norm_wavenumd()[q]['w_prime']))
            Power_dist[q] = {'P_core':P_core, 'P_sub':P_sub, 'P_clad':P_clad}
        return Power_dist
    
    def Electric_fielda(self):
        """
        Calculates the electric field distribution in the a dimension
        """
        dimension = self.a
        field_dist = OrderedDict()
        X_Y = []
        X = []
        E_y = []
        k = self.k()
        a = self.norm_wavenuma()  
        b = self.Aa()
        for p,A in b.items():
            kappa = a[p]['u'] / dimension
            xi = a[p]['w'] / dimension
            sigma = a[p]['w_prime'] / dimension
            phi = a[p]['phi']
            for x in np.linspace(-1.2*dimension,1.2*dimension,201):
                if x<-dimension:
                    field = A * np.cos(kappa*dimension+phi) * np.exp(xi*(x+dimension))
                    X_Y.append((x,field))
                elif -dimension<=x<=dimension:
                    field = A * np.cos(kappa*x-phi)
                    X_Y.append((x,field))
                elif x>dimension:
                    field = A * np.cos(kappa*dimension-phi) * np.exp(-sigma*(x-dimension))
                    X_Y.append((x,field))
            for elements in X_Y:
                X.append(elements[0])
                E_y.append(float(elements[1]))
            field_dist[p] = {'x':X, 'E_y':E_y} 
            X = []
            E_y = []
            X_Y = []
        return field_dist
   
                    
            
            


In [339]:
#All parameters mode 0, dimension a
file = planar_waveguide()
delta = file.delta()
n_1 = file.n_1
n_0 = file.n_0
n_s = file.n_s
a = file.a
k = file.k()
w_freq = file.ang_freq()
v = file.va()
gamma = file.gamma()
b = file.ba()[0]
n_e = file.n_ea()[0]
beta = file.betaa()[0]
A = file.Aa()[0]
u = file.norm_wavenuma()[0]['u']
w = file.norm_wavenuma()[0]['w']
w_prime = file.norm_wavenuma()[0]['w_prime']
phi = file.norm_wavenuma()[0]['phi']
E_y = file.Electric_fielda()[0]


In [340]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(E_y['x'],E_y['E_y'], color = 'm', label = 'Mode 0')
#plt.plot(np.resize(a[1]['x'],(200,1)),np.resize(a[1]['E_y'],(200,1)), color = 'b', label = 'Mode 1')
#plt.plot(np.resize(a[2]['x'],(200,1)),np.resize(a[2]['E_y'],(200,1)), color = 'g', label = 'Mode 2')
plt.xlabel('x')
plt.ylabel('E_y')
plt.legend(loc = 0)
plt.grid(True)
plt.show()

In [338]:

beta1 = beta                #Effective index * k  page 
beta2 = k * n_1 * np.cos(phi)  #page 
xi1 = w/a                      #page
xi2 = np.sin(phi)/(np.sqrt(2*delta)) #page 16
kappa1 = (u/a)
kappa2 = np.sqrt(k**2 * n_1**2 - beta2**2)
print(xi1)
print(xi2)
print(kappa1)
print(kappa2/2)
#Problem beta1  is different from beta2, kappa1 from kappa2

2718140.30549
2.34505257875
359799.463421
3214891.41001


'print(kappa1)\nprint(kappa2)\nprint(beta1)\nprint(beta2)'

In [ ]:
dimension = self.a
field_dist = OrderedDict()
X_Y = []
X = []
E_y = []
k = self.k()
a = self.norm_wavenuma()  
b = self.Aa()
for p,A in b.items():
    kappa = a[p]['u'] / dimension
    xi = a[p]['w'] / dimension
    sigma = a[p]['w_prime'] / dimension
    phi = a[p]['phi']
    for x in np.linspace(-1.2*dimension,1.2*dimension,201):
                if x<-dimension:
                    field = A * np.cos(kappa*dimension+phi) * np.exp(xi*(x+dimension))
                    X_Y.append((x,field))
                elif -dimension<=x<=dimension:
                    field = A * np.cos(kappa*x-phi)
                    X_Y.append((x,field))
                elif x>dimension:
                    field = A * np.cos(kappa*dimension-phi) * np.exp(-sigma*(x-dimension))
                    X_Y.append((x,field))
            for elements in X_Y:
                X.append(elements[0])
                E_y.append(float(elements[1]))
            field_dist[p] = {'x':X, 'E_y':E_y} 
            X = []
            E_y = []
            X_Y = []